In [ ]:
#Issues to address
#figure out optimal stride and text_length
#

In [1]:
import pinecone
from tqdm import tqdm
from dotenv import load_dotenv
import os

# Load environment variables from .env file
#overide = true just forces a reload on the .env file in case api key changes
load_dotenv(override=True)

# Access the API key
api_key = os.getenv("PINECONE_API_KEY")


C:\Users\Brady\.vscode\omnistudy-AI_Testing\venv\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
#Initialize the pinecone index
import pinecone      

pinecone.init(      
	api_key=api_key,      
	environment='gcp-starter'      
)      
index = pinecone.Index('haystack')

In [3]:
#load a pinecone document store object with the index defined previously
from haystack.document_stores import PineconeDocumentStore

document_store = PineconeDocumentStore(
    api_key=api_key,
    pinecone_index=index,
    similarity="cosine",
    embedding_dim=768
)

In [9]:
print(document_store.index)
print(document_store.embedding_dim)
print(document_store.type)

document
768
PineconeDocumentStore


In [5]:
#Textbook Extraction to make digestable for docu store
from TextBookExctraction import Process_PDF
#max_chunk_length is the max token length of each vector within the database
#stride refers to the step taken to find the middle of each vector. 
#If stride is 2 and if max_length is 3, we move 2 steps forwards and each vector will contain 3 tokens with an overlap of 1
# [1,2,3] , [3,4,5], [5,6,7], ... , [n-1,n,n+1]            with each array referring to a chunk/vector
pdf_processor = Process_PDF(pdf_path="./Textbooks/CrackingTheCodingInterview.pdf")
text = pdf_processor.extract_text_from_pdf()
cleaned_text = pdf_processor.preprocess_text(text)
text_chunks = pdf_processor.segment_text(cleaned_text, max_chunk_length=500, stride=400)
print(len(text_chunks))

7897


In [6]:
import torch
#Initialize retriever model
from haystack.nodes import EmbeddingRetriever
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
    model_format="sentence_transformers"
)


C:\Users\Brady\.vscode\omnistudy-AI_Testing\venv\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
from haystack import Document

batch_size = 256
total_doc_count = len(text_chunks)

counter = 0
docs = []
for d in text_chunks:
    doc = Document(
        content = d
    )
    docs.append(doc)
    counter += 1
    if counter % batch_size == 0 or counter == total_doc_count:
        embeds = retriever.embed_documents(docs)
        for i, doc in enumerate(docs):
            doc.embedding = embeds[i]
        document_store.write_documents(docs)
        docs.clear()
    if counter == total_doc_count:
        break


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Writing Documents: 100%|████████████████████████████████████████████████████████████| 256/256 [00:01<00:00, 177.57it/s]


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Writing Documents: 100%|████████████████████████████████████████████████████████████| 256/256 [00:00<00:00, 336.79it/s]


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

...

Writing Documents: 100%|████████████████████████████████████████████████████████████| 256/256 [00:00<00:00, 333.79it/s]


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Writing Documents: 100%|████████████████████████████████████████████████████████████| 256/256 [00:00<00:00, 332.99it/s]


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Writing Documents: 256it [00:00, 379.14it/s]                                                                           


In [14]:
#example usage of retriever
from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents

search_pipe = DocumentSearchPipeline(retriever)
result = search_pipe.run(
    query="what happens during a coding interview?",
    params={"Retriever": {"top_k": 2}}
)

print_documents(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: what happens during a coding interview?

{   'content': '(and many other companies). algorithm and coding problems form '
               'the \n'
               'largest component of the interview process. Think of these as '
               'problem-solving questions. The interviewer \n'
               'is looking to evaluate your ability to solve algorithmic '
               "problems you haven't seen before. \n"
               'Very often, you might get through only one question in an '
               'interview. Forty-five minutes is not a long time, \n'
               "and it's difficult to get through several different questions "
               'in that time frame. \n'
               'You should do your best to talk out loud t',
    'name': None}

{   'content': "s to explore what areas of technology you're familiar with. \n"
               "Next, you fly to Seattle (or whichever office you're "
               'interviewing for) for four or five interviews with one or \n'

In [63]:
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

rag_prompt = PromptTemplate(
    prompt="""Synthesize a comprehensive answer from the following text for the given question.
                             Provide a clear and concise response that summarizes the key points and information presented in the text.
                             Your answer should be in your own words and be no longer than 50 words.
                             \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
    output_parser=AnswerParser(),
)

prompt_node = PromptNode(model_name_or_path="vblagoje/bart_lfqa", default_prompt_template=rag_prompt)


In [69]:
from haystack.pipelines import Pipeline
pipeline = Pipeline()

pipeline.add_node(component=retriever, name="retriever", inputs=["Query"])
pipeline.add_node(component=prompt_node, name="prompt_node", inputs=["retriever"])

# result = pipeline.run(query="what happens during a coding interview?")


In [70]:
output = pipeline.run(query="What happens during a coding interview?")
print(output)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1303 > 1024). Running this sequence through the model will result in indexing errors
The prompt has been truncated from 1303 tokens to 924 tokens so that the prompt length and answer length (100 tokens) fit within the max token limit (1024 tokens). Shorten the prompt to prevent it from being cut off


{'answers': [<Answer {'answer': "I'm a software engineer. I'm not sure if this is the right place to ask this, but I'll give it a shot. I'm not sure if this is the right place to ask this, but I'll give it a shot. I'm a software engineer. I'm a software engineer. I'm a software engineer. I'm a software engineer. I'm a software engineer. I'm a software engineer. I'm a software engineer. I'm a software engineer.", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['3a4a4200b73a0aab07f1c7cbd9caa735', 'd6335099aa506c5c6556ad120d71bda7', 'ef538cb8f7600ec9389b8e4257182ec7', '63e845fc453e406698cb3748c1adc8dd', '66dc5042a398f073352886020361d0f3', '2ba63f7121713711c084f0b0b1daaaa', '9d768da7a47a48a6476247d592056616', '5240ee47f4c7b159c340266116528e08', '9bedc26b6b4bedb7bfe14f0fdbc5f0cc', '385cbae114f8e72634ae2cb0b581c7d0'], 'meta': {'prompt': 'Synthesize a comprehensive answer from the following text for the given ques

In [68]:
result_pipeline = pipeline.run(
    query="who wrote this book?",
    params={
        "Retriever": {"top_k": 3},
        "prompt_node": {"top_k": 1} 
    }
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Exception: Exception while running node 'prompt_node': Invalid parameter 'top_k' for the node 'prompt_node'.
Enable debug logging to see the data that was passed when the pipeline failed.

In [66]:
print(result_pipeline["answers"][0])

<Answer: answer='I'm not sure if this is the right subreddit for this, but I'll give it a shot. I'm a software engineer, and I've been doing this for a while. I've been doing this for a while, and I've been doing it for a while. I'm not sure if this is the right subreddit for this, but I'll give it a shot. I'm going to try to explain this as best I can. I'm going to try to explain it', score=None, context=None>


In [42]:
# print(result["answers"][0].answer)
prin

I'm a software engineer. I'm not sure if this is the right place to ask this, but I'll give it a shot. I'm not sure if this is the right place to ask this, but I'll give it a shot. I'm a software engineer. I'm a software engineer. I'm a software engineer. I'm a software engineer. I'm a software engineer. I'm a software engineer. I'm a software engineer. I'm a software engineer.


In [22]:
pt = PromptTemplate("lfqa", "question: {query} context: {join(documents, delimiter='<P>')}")

res = p.prompt(prompt_template=pt, query=query, documents=[Document(d) for d in documents])

ImportError: cannot import name 'GenerativeQAPipeline' from 'haystack.pipelines' (C:\Users\Brady\.vscode\omnistudy-AI_Testing\venv\lib\site-packages\haystack\pipelines\__init__.py)